# Convolutional Neural Networks (Fashion MNIST)

In [179]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm

In [129]:
#hyperparameters
BATCH_SIZE=32
INPUT_SHAPE = (1, 1, 28, 28)
EPOCHS = 10
LR = 0.001

In [148]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Get and preprocess the data

In [7]:
train_dataset = torchvision.datasets.FashionMNIST('./data', 
                                                  transform=transforms.ToTensor(), 
                                                  train=True, 
                                                  download=True)

In [13]:
test_dataset = torchvision.datasets.FashionMNIST('./data',
                                                transform=transforms.ToTensor(),
                                                train=False,
                                                download=True)

## Create the dataloader

In [25]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [26]:
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Create the model

In [159]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        #important: padding is applied before the convolution
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=2)
        
        #calculate the size for the input into the fully connected nn.Linear
        with torch.no_grad():
            dummy = torch.zeros(INPUT_SHAPE)
            self.flat_size = np.prod(self.cnn(dummy).shape)
        
        
        self.fc1 = nn.Linear(in_features=self.flat_size, out_features=100)
        self.fc2 = nn.Linear(in_features=100, out_features=10)
        
        self.to(DEVICE)
        
    def cnn(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        return x
        
    def forward(self, x):
        x = self.cnn(x)
        x = x.view(-1, self.flat_size)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [156]:
model = Model()

## Create the criterion and optimizer

In [161]:
criterion = nn.CrossEntropyLoss()

In [165]:
optimizer = optim.Adam(params=model.parameters(), lr=LR)

## Training loop

In [180]:
for epoch in range(EPOCHS):
    print(f'Starting epoch: {epoch+1}')
    for images, labels in tqdm(train_dataloader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        
        preds = model(images)
        loss = criterion(preds, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Starting epoch: 1


  0%|          | 0/1875 [00:00<?, ?it/s]

Starting epoch: 2


  0%|          | 0/1875 [00:00<?, ?it/s]

Starting epoch: 3


...

  0%|          | 0/1875 [00:00<?, ?it/s]

Starting epoch: 9


  0%|          | 0/1875 [00:00<?, ?it/s]

Starting epoch: 10


  0%|          | 0/1875 [00:00<?, ?it/s]

## Evaluation

In [188]:
n_images = 0
n_correct = 0

with torch.no_grad():
    for images, labels in tqdm(test_dataloader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        
        preds = torch.argmax(model(images), dim=1)
        n_images+=labels.shape[0]
        n_correct+= torch.eq(labels, preds).sum().item()
    accuracy = 100 * n_correct / n_images

  0%|          | 0/313 [00:00<?, ?it/s]

In [189]:
print(accuracy)

91.23
